Autor: Esteban Suárez Calvo

In [1]:
from datetime import timedelta
from time import time

from qiskit import QuantumCircuit
from qiskit.circuit.library import (
    RealAmplitudes,
    n_local,
    z_feature_map,
    zz_feature_map,
)
from qiskit_aer import AerSimulator
from qiskit_algorithms.optimizers import Optimizer
from qiskit_ibm_runtime import Sampler
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.optimizers import COBYLA, CRS, SPSA
from qiskit_machine_learning.utils import algorithm_globals
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

# CCAM - Práctica 2: Redes Neuronales Cuánticas

Antes de empezar, fijaremos la semilla aleatoria, de manera que los resultados obtenidos en el notebook sean reproducibles.

In [2]:
algorithm_globals.random_seed = 1

## Preprocesado

### Normalización del dataset

Normalizamos el dataset utilizando la normalización MinMax.

In [3]:
data = load_breast_cancer()
X = data.data
y = data.target
X = MinMaxScaler().fit_transform(X)
print(f"Número de características = {X.shape[1]}")

Número de características = 30


### Reducción de dimensionalidad

Reducimos la dimensionalidad del dataset utilizando una reducción PCA. De no hacerlo, necesitaríamos un total de 30 qubits, pues tenemos 30 características, y cada una se representa con un qubit. El problema está en que, aunque 30 qubits pueden no parecer demasiados, puede suponer un costo computacional bastante elevado el tener que emularlos.

Sin embargo, reducir la dimensionalidad tiene también su parte negativa, y es que al reducirla, estas dimensiones colapsan en menos, lo que hace que las clases que, anteriormente, podrían ser separables, puedan ya no serlo, por lo que debemos de ser cuidadosos y no pasarnos al reducir las dimensiones.

In [4]:
X = PCA(n_components=12).fit_transform(X)
number_of_features = X.shape[1]
print(f"Nuevo número de características = {number_of_features}")

Nuevo número de características = 12


### Dividir el conjunto entre entrenamiento y test

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, y, test_size=0.20, random_state=algorithm_globals.random_seed
)
print(f"Datos de entrenamiento: {len(X_train)}")
print(f"Datos de test: {len(X_test)}")

Datos de entrenamiento: 455
Datos de test: 114


### Clasificación con un SVM

Para comprobar que el problema es resoluble con la reducción de dimensionalidad que hemos aplicado antes, utilizamos un SVM

In [6]:
svc = SVC()
svc.fit(X_train, Y_train)

train_score = svc.score(X_train, Y_train)
test_score = svc.score(X_test, Y_test)

print(f"SVC en entrenamiento: {train_score}")
print(f"SVC en test: {test_score}")

SVC en entrenamiento: 0.9824175824175824
SVC en test: 0.9736842105263158


## Desarrolla una QNN para predecir la clase de cada caso

A continuación, definimos una función que, dado un `feature_map`, un `ansatz` y un `optimizer` arbitrarios, crea el `VQC` correspondiente y lo entrena.

Definimos también un callback para hacer más interactivo el proceso de entrenamiento, de manera que, a cada iteración, se muestre el número de la iteración y el *loss*:

In [7]:
def train_VQC(
    feature_map: QuantumCircuit, ansatz: QuantumCircuit, optimizer: Optimizer
) -> VQC:
    global X_train, Y_train

    number_of_features = X.shape[1]
    feature_map = zz_feature_map(number_of_features)

    vqc = VQC(
        feature_map=feature_map,
        ansatz=ansatz,
        optimizer=optimizer,
        callback=LossLogger().log_loss,
        sampler=Sampler(AerSimulator()),
    )

    start = time()
    vqc.fit(X_train, Y_train)
    total_time = time() - start
    print(f"Tiempo total de entrenamiento: {timedelta(seconds=total_time)}")

    return vqc


class LossLogger:
    def __init__(self):
        self._loss_history = []

    def log_loss(self, weights, obj_func_eval):
        self._loss_history.append(obj_func_eval)

        iteration = len(self._loss_history)

        print(f"Iteración {iteration} - Loss: {obj_func_eval:.4f}")

Definimos también una función para poder evaluar el clasificador.

In [8]:
def evaluate_VQC(vqc: VQC) -> None:
    global X_train, Y_train

    train_score = vqc.score(X_train, Y_train)
    print(f"SVC en entrenamiento: {train_score:.5}")

    test_score = vqc.score(X_test, Y_test)
    print(f"SVC en test: {test_score:.5}")

## Ejecución con distintos `VQC`

Ahora que tenemos una función para crear el `VQC` y para evaluarlo, podemos realizar pruebas con distintos clasificadores y ver que tan bien funcionan.

Hemos realizado diversas pruebas, a continuación mostraremos algunas de ellas:

- **VQC 1**

In [9]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 1.0023
Iteración 2 - Loss: 0.9983
Iteración 3 - Loss: 1.0004
Iteración 4 - Loss: 0.9853
Iteración 5 - Loss: 0.9817
Iteración 6 - Loss: 0.9839
Iteración 7 - Loss: 0.9804
Iteración 8 - Loss: 0.9800
Iteración 9 - Loss: 0.9804
Iteración 10 - Loss: 0.9834
Iteración 11 - Loss: 0.9873
Iteración 12 - Loss: 0.9855
Iteración 13 - Loss: 0.9830
Iteración 14 - Loss: 0.9813
Iteración 15 - Loss: 1.0110
Iteración 16 - Loss: 0.9834
Iteración 17 - Loss: 0.9812
Iteración 18 - Loss: 0.9849
Iteración 19 - Loss: 0.9806
Iteración 20 - Loss: 0.9799
Tiempo total de entrenamiento: 0:04:30.873468
SVC en entrenamiento: 0.55604
SVC en test: 0.57895


- **VQC 2**

In [10]:
feature_map = z_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 0.9909
Iteración 2 - Loss: 0.9850
Iteración 3 - Loss: 0.9908
Iteración 4 - Loss: 0.9695
Iteración 5 - Loss: 0.9719
Iteración 6 - Loss: 0.9715
Iteración 7 - Loss: 0.9741
Iteración 8 - Loss: 0.9698
Iteración 9 - Loss: 0.9720
Iteración 10 - Loss: 0.9738
Iteración 11 - Loss: 0.9729
Iteración 12 - Loss: 0.9741
Iteración 13 - Loss: 0.9734
Iteración 14 - Loss: 0.9870
Iteración 15 - Loss: 0.9981
Iteración 16 - Loss: 0.9778
Iteración 17 - Loss: 0.9791
Iteración 18 - Loss: 0.9770
Iteración 19 - Loss: 0.9773
Iteración 20 - Loss: 0.9746
Tiempo total de entrenamiento: 0:04:29.397521
SVC en entrenamiento: 0.60879
SVC en test: 0.68421


- **VQC 3**

In [11]:
feature_map = zz_feature_map(number_of_features)
ansatz = RealAmplitudes(number_of_features).decompose()
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 0.9901
Iteración 2 - Loss: 0.9930
Iteración 3 - Loss: 0.9971
Iteración 4 - Loss: 0.9951
Iteración 5 - Loss: 0.9980
Iteración 6 - Loss: 0.9970
Iteración 7 - Loss: 0.9888
Iteración 8 - Loss: 0.9991
Iteración 9 - Loss: 0.9925
Iteración 10 - Loss: 0.9942
Iteración 11 - Loss: 0.9975
Iteración 12 - Loss: 0.9942
Iteración 13 - Loss: 0.9944
Iteración 14 - Loss: 1.0002
Iteración 15 - Loss: 0.9945
Iteración 16 - Loss: 0.9918
Iteración 17 - Loss: 0.9959
Iteración 18 - Loss: 0.9960
Iteración 19 - Loss: 0.9953
Iteración 20 - Loss: 0.9936
Tiempo total de entrenamiento: 0:04:30.139605
SVC en entrenamiento: 0.48571
SVC en test: 0.51754


- **VQC 4**

In [12]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear", reps=3)
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 0.9925
Iteración 2 - Loss: 0.9984
Iteración 3 - Loss: 0.9996
Iteración 4 - Loss: 0.9913
Iteración 5 - Loss: 0.9872
Iteración 6 - Loss: 0.9888
Iteración 7 - Loss: 0.9862
Iteración 8 - Loss: 0.9908
Iteración 9 - Loss: 0.9860
Iteración 10 - Loss: 0.9858
Iteración 11 - Loss: 0.9903
Iteración 12 - Loss: 0.9839
Iteración 13 - Loss: 0.9843
Iteración 14 - Loss: 0.9825
Iteración 15 - Loss: 0.9879
Iteración 16 - Loss: 0.9747
Iteración 17 - Loss: 0.9768
Iteración 18 - Loss: 0.9752
Iteración 19 - Loss: 0.9771
Iteración 20 - Loss: 0.9763
Tiempo total de entrenamiento: 0:04:30.274279
SVC en entrenamiento: 0.58022
SVC en test: 0.59649


- **VQC 5**

In [14]:
feature_map = z_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear", reps=3)
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 0.9982
Iteración 2 - Loss: 1.0091
Iteración 3 - Loss: 0.9998
Iteración 4 - Loss: 0.9939
Iteración 5 - Loss: 0.9917
Iteración 6 - Loss: 0.9952
Iteración 7 - Loss: 0.9957
Iteración 8 - Loss: 0.9894
Iteración 9 - Loss: 0.9895
Iteración 10 - Loss: 0.9951
Iteración 11 - Loss: 0.9878
Iteración 12 - Loss: 0.9924
Iteración 13 - Loss: 0.9929
Iteración 14 - Loss: 0.9992
Iteración 15 - Loss: 1.0108
Iteración 16 - Loss: 0.9836
Iteración 17 - Loss: 0.9899
Iteración 18 - Loss: 0.9883
Iteración 19 - Loss: 0.9862
Iteración 20 - Loss: 0.9911
Tiempo total de entrenamiento: 0:04:30.730952
SVC en entrenamiento: 0.52527
SVC en test: 0.63158


- **VQC 6**

In [9]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = CRS()

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Tiempo total de entrenamiento: 3:46:17.754241
SVC en entrenamiento: 0.6021978021978022
SVC en test: 0.6228070175438597


- **VQC 7**

In [14]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = SPSA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Tiempo total de entrenamiento: 0:20:46.916321
SVC en entrenamiento: 0.6153846153846154
SVC en test: 0.6491228070175439


## Resultados

Tenemos entonces los siguientes resultados:


| Clasificador | `feature_map`    | `ansatz`          | `optimizer` | Accuracy en entrenamiento | Accuracy en test | Tiempo de entrenamiento |
| ------------ | ---------------- | ----------------- | ----------- | ------------------------- | ---------------- | ----------------------- |
| **VQC 1**    | `zz_feature_map` | `n_local`         | `COBYLA`    | 0.55604                   | 0.57895          | 0:04:30                 |
| **VQC 2**    | `z_feature_map`  | `n_local`         | `COBYLA`    | 0.60879                   | 0.68421          | 0:04:29                 |
| **VQC 3**    | `zz_feature_map` | `RealAmplitudes`  | `COBYLA`    | 0.48571                   | 0.51754          | 0:04:30                 |
| **VQC 4**    | `zz_feature_map` | `n_local(reps=3)` | `COBYLA`    | 0.58022                   | 0.59649          | 0:04:30                 |
| **VQC 5**    | `z_feature_map`  | `n_local(reps=3)` | `COBYLA`    | 0.52527                   | 0.63158          | 0:04:30                 |
| **VQC 6**    | `zz_feature_map` | `n_local`         | `CRS`       | 0.60219                   | 0.62280          | 3:46:17                 |
| **VQC 7**    | `zz_feature_map` | `n_local`         | `SPSA`      | 0.61538                   | 0.64912          | 0:20:46                 |


## Conclusiones

En la siguiente tabla podemos observar una comparativa entre los distintos clasificadores que hemos probado:


Independientemente del `feature_map`, `ansatz` u `optimizer` escogido, y a pesar de que algunos clasificadores funcionan mejor que otros, ninguno de los clasificadores es capaz de aprender, ni el conjunto de entrenamiento ni el de test.

A pesar de que en algún caso llegamos a clasificar correctamente casi el 65% del conjunto de test, independientemente del número de iteraciones que realicemos, el loss se mantiene prácticamente igual.

Por lo tanto, dado que ninguno de los clasificadores es capaz de funcionar mínimamente bien, todo parece indicar que un VQC es una red demasiado compleja como para utilizarla en un problema tan sencillo como este, pues vemos que un clasificador clásico, como un SVC, es capaz de clasificar correctamente prácticamente todo el conjunto de test, tardando pocos segundos en ser entrenado, mientras que los VQC tardan decenas de minutos, o incluso horas.

Otro punto a considerar para estos malos resultados es la reducción de la dimensionalidad aplicada. Originalmente partimos de 30 características, pero para poder realizar el entrenamiento en tiempos más o menos razonables, nos vemos forzados a reducir considerablemente la dimensionalidad, pasando a 12 características. Como podemos observar, un clasificador clásico (en este caso hemos probado con SVC), es capaz de resolver el problema aún con tan pocas características. Sin embargo, los VQC tienden a funcionar mejor a medida que se aumenta el número de características, pero para poder probar un clasificador de ese estilo se necesitaría un hardware real bastante potente. Aun así, se han probado también 